In [3]:
import pandas as pd 
grouped_bbq = pd.read_pickle("grouped_bbq.pkl")
#one hot encode bbq type to see what bbqs are available at each group
grouped_bbq = pd.get_dummies(grouped_bbq,prefix=['bbq_type'], columns = ['bbq_type'], drop_first=False)

def create_group_id(df):
    if df['group'] >=0: return str(df['group'])
    else: return df['asset_id']
grouped_bbq["group_id"] = grouped_bbq.apply(create_group_id,axis=1)
grouped_bbq["count"] = 1 


In [4]:
#split multi and single bbqs
grouped_bbq_sum = grouped_bbq[["group_id","bbq_type_Electric","bbq_type_Gas","bbq_type_Wood","count"]].groupby("group_id", as_index=False).agg(["sum"])
grouped_bbq_avg = grouped_bbq[["group_id","latitude","longitude"]].groupby("group_id", as_index=False).agg(["mean"])


In [5]:
amenities = {"bbq": {"csv":'https://www.data.act.gov.au/resource/n3b4-mm52.csv', "colour":"red","name": "BBQ","id":"asset_id","attrib":"bbq_type"} , 
             "drink": {"csv":'https://www.data.act.gov.au/resource/8eg4-uskm.csv', "colour":"blue","name": "Bubbler","id":"id","attrib":""}, 
             "furn": {"csv":'https://www.data.act.gov.au/resource/ch39-bukk.csv', "colour":"orange","name": "Furniture","id":"asset_id","attrib":"feature_type"}, 
             "toilet": {"csv":'https://www.data.act.gov.au/resource/3tyf-txjn.csv', "colour":"brown","name": "Toilet","id":"asset_id","attrib":"toilet_type_text"} ,
             "fitness": {"csv":'https://www.data.act.gov.au/resource/h4qc-3txc.csv', "colour":"grey","name": "Fitness Equipment","id":"id","attrib":"type"} ,
             "playground": {"csv":'https://www.data.act.gov.au/resource/fwth-mr9q.csv', "colour":"yellow","name": "Playground","id":"asset_id","attrib":""} 
            }
for a in amenities:
    amenities[a]["df"] = pd.read_pickle(a+".pkl")

In [6]:
exclude = ["bbq","fitness"]
amenities_near = pd.DataFrame()
for a in amenities:
    if a not in exclude:        
        amenities_near= amenities_near.append(amenities[a]["df"].rename(columns={amenities[a]["id"]:"amen_id"})[["amen_id","cohort","latitude","longitude","x","y"]])
amenities_near = amenities_near.reset_index()


In [7]:
#cross join 
import numpy as np 
def cartesian_product_simplified(left, right):
    la, lb = len(left), len(right)
    ia2, ib2 = np.broadcast_arrays(*np.ogrid[:la,:lb])

    
    return pd.DataFrame(
        np.column_stack([left.values[ia2.ravel()], right.values[ib2.ravel()]]),
        columns=list(left.columns)+ list(right.columns)
        )


bbq_distance =\
cartesian_product_simplified(\
grouped_bbq.rename(columns={"latitude":"lat1","longitude":"lon1"})[["group_id","lat1","lon1"]]\
,amenities_near.rename(columns={"latitude":"lat2","longitude":"lon2"})[["amen_id","cohort","lat2","lon2"]])

bbq_distance = bbq_distance.reset_index()

from math import radians, cos, sin, asin, sqrt

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    #print(lon1, lat1, lon2, lat2)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

from haversine import haversine

#bbq_distance = bbq_distance.drop(columns=["index"])

bbq_distance["distance"] = haversine_np(bbq_distance['lon1'].astype(float),bbq_distance['lat1'].astype(float),bbq_distance['lon2'].astype(float),bbq_distance['lat2'].astype(float))* 1000

In [8]:
#get all amenities close to BBQ groups
bbq_amenity = bbq_distance[bbq_distance["distance"]<=100][["group_id","cohort"]]
bbq_amenity = pd.get_dummies(bbq_amenity,prefix=['n'], columns = ['cohort'], drop_first=False)
bbq_amenity_sum = bbq_amenity.groupby("group_id", as_index=False).agg(["sum"])


In [199]:
#print(bbq_amenity["group_id"].unique())

In [9]:
bbq_amen_2  = grouped_bbq_sum.join(bbq_amenity_sum, on=["group_id"])
bbq_amen_2 = bbq_amen_2.join(grouped_bbq_avg, on=["group_id"])
bbq_amen_2.update(bbq_amen_2[[ "n_drink", "n_furn", "n_playground", "n_toilet"]].fillna(0))
bbq_amen_2["amen_count"] = bbq_amen_2[ ["n_drink", "n_furn", "n_playground", "n_toilet"]].sum(axis=1)
#Bokeh maps are in mercator. Convert lat lon fields to mercator units for plotting

def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df


bbq_amen_2=wgs84_to_web_mercator(bbq_amen_2,'longitude','latitude')
bbq_amen_2 = bbq_amen_2.reset_index()


In [13]:
from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

In [10]:
scale=100
x=bbq_amen_2['x']
y=bbq_amen_2['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.

x_min=int(x.mean() - (scale * 150))
x_max=int(x.mean() + (scale * 150))
y_min=int(y.mean() - (scale * 150))
y_max=int(y.mean() + (scale * 150))

In [24]:
#rename to first of each tuple
bbq_amen_2.columns = bbq_amen_2.columns.get_level_values(0)
bbq_amen_2
bbq_types=['bbq_type_Electric','bbq_type_Gas','bbq_type_Wood']
amen_types=['n_drink','n_furn','n_playground','n_toilet']

,group_id,bbq_type_Electric,bbq_type_Gas,bbq_type_Wood,count,n_drink,n_furn,n_playground,n_toilet,latitude,longitude,amen_count,x,y
0,0,1,2,28,31,0.0,287.0,0.0,22.0,-35.325832,148.947444,309.0,1.658075e+07,-4.208249e+06
1,1,0,0,13,13,0.0,73.0,0.0,9.0,-35.244645,148.951235,82.0,1.658118e+07,-4.197177e+06
2,10,10,0,0,10,0.0,242.0,0.0,4.0,-35.323236,148.941544,246.0,1.658010e+07,-4.207895e+06
3,11,0,2,13,15,0.0,60.0,0.0,13.0,-35.662249,148.988713,73.0,1.658535e+07,-4.254246e+06
4,12,0,0,4,4,0.0,0.0,0.0,0.0,-35.444296,148.923768,0.0,1.657812e+07,-4.224424e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,BBQ76,0,1,0,1,1.0,0.0,0.0,0.0,-35.298719,149.110620,1.0,1.659892e+07,-4.204550e+06
115,BBQ80,0,1,0,1,1.0,0.0,0.0,0.0,-35.295686,149.095155,1.0,1.659720e+07,-4.204137e+06
116,BBQ85,0,1,0,1,0.0,0.0,0.0,0.0,-35.290275,149.092152,0.0,1.659686e+07,-4.203399e+06
117,BBQ89,1,0,0,1,0.0,0.0,0.0,0.0,-35.237871,149.074260,0.0,1.659487e+07,-4.196254e+06


In [23]:

#Defining the map tiles to use. I use OSM, but you can also use ESRI images or google street maps.

tile_provider=get_provider(Vendors.CARTODBPOSITRON_RETINA)

#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='Canberra Amenities',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator',
    width=500,
    output_backend="webgl"
    )

plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False
plot.title.text_font_size="20px"

output_notebook()
#function takes a column to determine radius and the dataframe with converted mercator coordinates to create a bubble map. 
def bubble_map(plot,df,radius_col,lon,lat,scale,color='orange',leg_label='Bubble Map'):

  df['radius']=scale
    
  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius='radius',legend_label=leg_label,hover_color='red')

  tip_label='@'+radius_col
  lat_label='@'+lat
  lon_label='@'+lon

  circle_hover = HoverTool(tooltips=[(radius_col,tip_label),('Lat:',lat_label),('Lon:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.
for a in amenities:
    if a not in exclude:        
        bubble_map(plot=plot,
                   df=amenities[a]["df"],
                   radius_col='location', 
                   leg_label=amenities[a]["name"],
                   lon='longitude',
                   lat='latitude',
                   color = amenities[a]["colour"],
                   scale=10)

def bbq_map(plot,df,lon,lat,group,scale,color='red',leg_label='Bbq Map',radius=50):

  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius=radius,legend_label=leg_label,hover_color='red')

  lat_label='@'+lat
  lon_label='@'+lon
  amen_label='@'+group

  circle_hover = HoverTool(tooltips=[('Toilets:',amen_label),('Lat:',lat_label),('Lon:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.

bbq_map(plot=plot,
           df=bbq_amen_2,
           leg_label='bbq_type_Gas',
           group='n_toilet',
           lon='longitude',
           lat='latitude',
           scale=100)

sitesource=ColumnDataSource(bbq_amen_2)
# Make a slider object to toggle the month shown
slider = Slider(title = 'Month', start = 1, end = 12,step = 1, value = 1)
# This callback triggers the filter when the slider changes
callback = CustomJS(args = dict(source=sitesource), 
                    code = """source.change.emit();""")
slider.js_on_change('value', callback)
# Creates custom filter that selects the rows of the month based on the value in the slider
custom_filter = CustomJSFilter(args = dict(slider = slider, 
                                           source = sitesource), 
                               code = """
var indices = [];
// iterate through rows of data source and see if each satisfies some constraint
for (var i = 0; i < source.get_length(); i++){
 if (source.data[‘Month’][i] == slider.value){
 indices.push(true);
 } else {
 indices.push(false);
 }
}
return indices;
""")# Uses custom_filter to determine which set of sites are visible
view = CDSView(source = sitesource, filters = [custom_filter])

plot.toolbar.active_scroll=plot.select_one(WheelZoomTool)
# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(plot, widgetbox(slider))

output_file("./site/maps.html")
save(layout)

Loading BokehJS ...

'/home/chez/projects/where_to_bbq/site/maps.html'

In [21]:
print(dir(plot.toolbar))


['__cached_all__overridden_defaults__', '__cached_all__properties__', '__cached_all__properties_with_refs__', '__class__', '__container_props__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__properties__', '__properties_with_refs__', '__qualified_model__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__view_model__', '__view_module__', '__weakref__', '_attach_document', '_callbacks', '_clear_extensions', '_clone', '_detach_document', '_document', '_event_callbacks', '_id', '_overridden_defaults', '_property_values', '_repr_html_', '_temp_document', '_to_json_like', '_trigger_event', '_unstable_default_values', '_unstable_themed_values', '_update_event_callbacks', 'active_drag', 'active_inspect', 'active_multi', 'active_scroll', 'active_tap', 'apply_theme', 'autohide', 'dataspecs',